In [1]:
# Import the libraries

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [4]:
# Readinf our train dataset

train = pd.read_csv('train.csv')
train.head()


,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [9]:
#we will read all the training images, store them in a list, and finally convert that list into a numpy array

# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['filename'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 49000/49000 [01:30<00:00, 541.84it/s]


In [10]:
# one-hot encode the target variable.

y=train['label'].values
y = to_categorical(y)

In [11]:
# Splitting into train and validation 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [12]:
# Create a simple architecture with 2 convolutional layers, one dense hidden layer and an output layer.

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [13]:
# Complie and fit 

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [14]:
'''
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 39200 samples, validate on 9800 samples
Epoch 1/10
39200/39200 [==============================] - 140s 4ms/step - loss: 0.2373 - accuracy: 0.9267 - val_loss: 0.0695 - val_accuracy: 0.9806
Epoch 2/10
39200/39200 [==============================] - 135s 3ms/step - loss: 0.0941 - accuracy: 0.9718 - val_loss: 0.0567 - val_accuracy: 0.9840
Epoch 3/10
39200/39200 [==============================] - 133s 3ms/step - loss: 0.0707 - accuracy: 0.9775 - val_loss: 0.0481 - val_accuracy: 0.9860
Epoch 4/10
39200/39200 [==============================] - 132s 3ms/step - loss: 0.0550 - accuracy: 0.9828 - val_loss: 0.0428 - val_accuracy: 0.9877
Epoch 5/10
39200/39200 [==============================] - 129s 3ms/step - loss: 0.0480 - accuracy: 0.9848 - val_loss: 0.0418 - val_accuracy: 0.9887
Epoch 6/10
39200/39200 [==============================] - 129s 3ms/step - loss: 0.0420 - accuracy: 0.9864 - val_loss: 0.0399 - val_accuracy: 0.9890
Epoch 7/10
39200/39200 [==============================] - 132s 

In [26]:
# Increasing the number of epochs for better results
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Train on 39200 samples, validate on 9800 samples
Epoch 1/30
39200/39200 [==============================] - 142s 4ms/step - loss: 0.0232 - accuracy: 0.9926 - val_loss: 0.0458 - val_accuracy: 0.9906
Epoch 2/30
39200/39200 [==============================] - 140s 4ms/step - loss: 0.0250 - accuracy: 0.9922 - val_loss: 0.0453 - val_accuracy: 0.9885
Epoch 3/30
39200/39200 [==============================] - 134s 3ms/step - loss: 0.0199 - accuracy: 0.9936 - val_loss: 0.0457 - val_accuracy: 0.9902
Epoch 4/30
39200/39200 [==============================] - 136s 3ms/step - loss: 0.0190 - accuracy: 0.9935 - val_loss: 0.0469 - val_accuracy: 0.9908
Epoch 5/30
39200/39200 [==============================] - 135s 3ms/step - loss: 0.0195 - accuracy: 0.9938 - val_loss: 0.0458 - val_accuracy: 0.9896
Epoch 6/30
39200/39200 [==============================] - 131s 3ms/step - loss: 0.0189 - accuracy: 0.9941 - val_loss: 0.0445 - val_accuracy: 0.9918
Epoch 7/30
39200/39200 [==============================] - 131s 

In [41]:
test = pd.read_csv('test.csv')
test.head()

,filename
0,49000.png
1,49001.png
2,49002.png
3,49003.png
4,49004.png


In [34]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['filename'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 21000/21000 [00:40<00:00, 520.53it/s]


In [35]:
# making predictions

prediction = model.predict_classes(test)

In [37]:

# creating submission file
sample = pd.read_csv('sample_submission.csv')
sample.head()




,filename,label


In [42]:
sample['filename'] = test['filename']
sample['label'] = prediction
sample.to_csv('samplebase2.csv', header=True, index=False)